# Adding Charts via API

This notebook walks through how we can use Fiddler's API to add charts to the platform.

Note that **this API is not officially supported** and lacks the same guarantees around backwards compatibility that our other APIs have.

## 1. Run the following code block

In [ ]:
import logging
import yaml
from uuid import uuid4

import fiddler as fdl
from fiddler.libs.http_client import RequestClient

logger = logging.getLogger(__name__)


def add_chart(project: fdl.Project, model: fdl.Model, unique_id: str, client: RequestClient, chart: dict):
    charts_url = '/v3/charts'
    title = f'{chart.get("title")}'
    chart['title'] = title

    # Handle performance/feature analytics and metric card charts
    if chart['query_type'] == 'ANALYTICS':
        model_name = chart['data_source']['model'].get('name')
        version = chart['data_source']['model'].get('version', 'v1')
        model = fdl.Model.from_name(
            name=model_name, project_id=project.id, version=version
        )
        dataset_name = chart['data_source'].get('env_id', None)
        if dataset_name:
            dataset = fdl.Dataset.from_name(name=dataset_name, model_id=model.id)
            chart['data_source'].update(
                {
                    'env_id': dataset.id,
                }
            )
            
        chart['data_source'].update(
            {
                'model': {'id': model.id, 'name': model_name},
            }
        )
    # Handle Monitoring and Embedding Charts
    else:
      for index, query in enumerate(chart['data_source']['queries']):
          version = query.get('version', 'v1')
          query.update(
              {
                  'model': {'id': model.id, 'name': model.name},
                  'model_name': model.name,
                  'version': version,
              }
          )

          baseline_name = query.get('baseline_name')
          if baseline_name:
              baseline = fdl.Baseline.from_name(name=baseline_name, model_id=model.id)
              baseline_id = baseline.id
              query['baseline_id'] = baseline_id
              del query['baseline_name']

          if query.get('metric_type') == 'custom':
              custom_metrics = fdl.CustomMetric.from_name(
                  name=query.get('metric'), model_id=model.id
              )
              query['metric'] = custom_metrics.id

          segment = query.get('segment')
          if segment:
              segment = fdl.Segment.from_name(name=segment, model_id=model.id)
              query['segment'] = {}
              query['segment']['id'] = segment.id

          chart['data_source']['queries'][index] = query

    chart['project_id'] = project.id
    client.post(url=charts_url, data=chart)

## 2. Enter your URL, auth token, project name, and model name

In [ ]:
URL = ''
AUTH_TOKEN = ''

PROJECT_NAME = ''
MODEL_NAME = ''

## 3. Initialize the client

In [ ]:
fdl.init(
    url=URL,
    token=AUTH_TOKEN
)

project = fdl.Project.from_name(PROJECT_NAME)

model = fdl.Model.from_name(
    project_id=project.id,
    name=MODEL_NAME
)

client = RequestClient(
    base_url=URL,
    headers={
        'Authorization': f'Bearer {AUTH_TOKEN}',
        'Content-Type': 'application/json'
    }
)

## 4. Generate a unique ID for the new chart

In [ ]:
chart_id = uuid4()

## 5. Paste the response data object for a chart

This can be found by going to the Network tab of the developer tools in your browser and clicking Save in the UI (to save the chart).

You can then search for **"chart"** in the Network tab to find the POST request.

Then copy the data object from the response payload and paste it into the `chart` variable below.

**Note:** Some of the fields in this object (project/model/version names, etc.) are overridden by the `add_chart` code. Feel free to modify `add_chart` to meet your requirements.

In [ ]:
chart = {
    "id": "8f22a9e9-ca07-4e01-bc95-a40196f8a364",
    "organization": {
        "id": "febfb97f-6e51-4a43-a66c-86bab1c0c6fb",
        "name": "preprod"
    },
    "project": {
        "id": "0dde103e-bee5-4cbf-b28f-954b99cb6f32",
        "name": "py_test"
    },
    "title": "Sentiment Frequency Chart",
    "query_type": "MONITORING",
    "description": "Chart to track Sentiment",
    "options": {
        "queryNames": {
            "b842b54f-2af4-49d7-9fee-158c62d5ae3ds": ""
        },
        "customYAxis": [
            {
                "scale": "value",
                "query_keys": [
                    "b842b54f-2af4-49d7-9fee-158c62d5ae3ds"
                ]
            },
            {
                "scale": "value",
                "query_keys": []
            }
        ]
    },
    "data_source": {
        "query_type": "MONITORING",
        "filters": {
            "time_label": "7d",
            "time_range": {
                "start_time": "2025-04-09T00:00:00",
                "end_time": "2025-04-15T23:59:59"
            },
            "bin_size": "Day",
            "time_zone": "UTC"
        },
        "queries": [
            {
                "query_key": "b842b54f-2af4-49d7-9fee-158c62d5ae3ds",
                "model": {
                    "id": "173c3520-993d-47e0-bb43-f9e27cdf6e2f",
                    "name": "fiddler_rag_llm_chatbot"
                },
                "columns": [
                    "fdl_enrichment_qa_sentiment__question__sentiment"
                ],
                "metric": "frequency",
                "categories": [
                    "negative",
                    "neutral",
                    "positive"
                ],
                "metric_type": "statistic",
                "segment": {},
                "viz_type": "line"
            }
        ]
    },
    "created_at": "2025-04-14T08:20:54.794624+00:00",
    "updated_at": "2025-04-15T14:04:46.349927+00:00",
    "created_by": {
        "id": "9828ca24-9210-4963-b711-b846584d8f1e",
        "full_name": "User",
        "email": "user@fiddler.ai"
    },
    "updated_by": {
        "id": "9828ca24-9210-4963-b711-b846584d8f1e",
        "full_name": "User",
        "email": "user@fiddler.ai"
    }
}

## 6. Run `add_chart` to add the chart to Fiddler

In [ ]:
add_chart(project, model, chart_id, client, chart)